In [1]:
import keras
from keras.models import Model
import numpy as np
from keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.applications.resnet50 import decode_predictions, preprocess_input
from keras.models import load_model
import PIL.Image as Image
import os
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import glob
import numpy as np
import pandas as pd
import os
import shutil 
import matplotlib.pyplot as plt
from keras.preprocessing import image
from sklearn.preprocessing import LabelEncoder 
%matplotlib inline
model = load_model('D:\deepfake_detection\datasets\\new_densenet-121\\new_densenet-121-acc-81.h5')

In [2]:
IMG_WIDTH=300
IMG_HEIGHT=300
IMG_DIM = (IMG_WIDTH, IMG_HEIGHT)

- Take equal no of real (train) and fake (test)
- Create dataframe

In [3]:
labels = pd.read_csv("D:\deepfake_detection\datasets\\out.csv", index_col=0)
all_frames = glob.glob('D:\deepfake_detection\datasets\\frames/*')
test_frames = glob.glob('D:\deepfake_detection\datasets\\test_imgs/*')
real_frames = []
fake_frames = []
for frame in all_frames:
    file_name = os.path.basename(frame)
    label = labels.loc[file_name][0]
    if label == 'REAL':
        real_frames.append((file_name, label))
for frame in test_frames[:len(real_frames)]:
    file_name = os.path.basename(frame)
    fake_frames.append((file_name, 'FAKE'))
combine_real_fake = real_frames + fake_frames
df = pd.DataFrame(combine_real_fake, columns=['file_name', 'label'])
df.set_index('file_name', inplace=True)
labelencoder = LabelEncoder()
df['label'] = labelencoder.fit_transform(df['label'])
df

,label
file_name,
abarnvbtwb_1.jpg,1
abarnvbtwb_10.jpg,1
abarnvbtwb_11.jpg,1
abarnvbtwb_12.jpg,1
abarnvbtwb_13.jpg,1
...,...
srfefmyjvt_10.jpg,0
srfefmyjvt_11.jpg,0
srfefmyjvt_12.jpg,0


In [4]:
test_imgs = [image.img_to_array(image.load_img('D:\deepfake_detection\datasets\\main_test_imgs\\'+img, target_size=IMG_DIM)) for img in df.index]
test_imgs = np.array(test_imgs)
img_seq = df.index
img_seq[:5]

Index(['abarnvbtwb_1.jpg', 'abarnvbtwb_10.jpg', 'abarnvbtwb_11.jpg',
       'abarnvbtwb_12.jpg', 'abarnvbtwb_13.jpg'],
      dtype='object', name='file_name')

In [5]:
test_datagen = image.ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow(test_imgs, df['label'], batch_size=30)

In [6]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
functional_1 (Functional)    (None, 82944)             7037504   
_________________________________________________________________
dense (Dense)                (None, 512)               42467840  
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 513       
Total params: 49,768,513
Trainable params: 42,731,009
Non-trainable params: 7,037,504
____________________________________

In [7]:
feature_network= Model(model.input, model.get_layer('dense_1').output)
feature_network.summary()


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
functional_1_input (InputLay [(None, 300, 300, 3)]     0         
_________________________________________________________________
functional_1 (Functional)    (None, 82944)             7037504   
_________________________________________________________________
dense (Dense)                (None, 512)               42467840  
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
Total params: 49,768,000
Trainable params: 42,730,496
Non-trainable params: 7,037,504
_________________________________________________________________


In [8]:
feature = feature_network.predict(test_generator)

In [9]:
file_label_df = df.reset_index()
features_df = pd.DataFrame(feature)
df_main = pd.concat([file_label_df, features_df], axis=1)
df_main.drop(0, inplace=True)
feature_file_name = 'D:\deepfake_detection\datasets\\new_densenet-121\\denseNet121.csv' # save file
df_main.to_csv(feature_file_name, index=False)
df_main.head(2)

,file_name,label,0,1,2,3,4,5,6,7,...,502,503,504,505,506,507,508,509,510,511
1,abarnvbtwb_10.jpg,1,0.012252,0.0,2.452108,0.725467,0.147997,0.0,0.307768,0.437981,...,0.000000,0.060367,0.281849,1.058227,0.0,0.198189,0.0,0.724373,0.0,0.0
2,abarnvbtwb_11.jpg,1,0.738508,0.0,1.667492,0.356716,0.063301,0.0,0.000000,0.000000,...,1.020212,0.000000,0.000000,3.055794,0.0,1.051151,0.0,0.532673,0.0,0.0


## ------------------- RUN FROM HERE --------------------------------------

## TSBTCn-ary code

In [12]:
from PIL import Image
import numpy as np
from numpy.core.fromnumeric import sort
from xlwt import Workbook
import os

wb = Workbook()

sheet1 = wb.add_sheet('Sheet 1', cell_overwrite_ok=True)
input_tsbtc = 6 # CHANGE

for i in range(1, input_tsbtc+1):
    sheet1.write(0, i, 'R' + str(i))
    sheet1.write(0, input_tsbtc+i, 'G'+str(i))
    sheet1.write(0, 2*input_tsbtc+i, 'B'+str(i))
sheet1.write(0, 3*input_tsbtc+1, 'Images')

result = []

def write_into_word(curr_dir):
    img_format = ['JPEG', 'TIFF', 'BMP']
    count = 1
    for imagePath in os.listdir(curr_dir):
           
        # open folder
        sheet1.write(count, 1, imagePath)

        # red color
        try:
            img = Image.open(imagePath)
            format = img.format
            if format not in img_format:
                continue
            red, green, blue = img.split()
        except:
            continue
        redarr = np.asarray(red)
        red_array_1d = redarr.flatten()
        sorted_arr = np.sort(red_array_1d)
        ll = len(sorted_arr)
        red_array = []
        for i in range(input_tsbtc):
            if i != input_tsbtc - 1:
                red_array.append(np.average(
                    sorted_arr[i*ll//input_tsbtc: (i+1)*ll//input_tsbtc]))
            else:
                red_array.append(np.average(sorted_arr[i*ll//input_tsbtc:]))
        for i in range(1, input_tsbtc+1):
            sheet1.write(count, i, red_array[i-1])
        # print(imagePath)

        # green color
        greenarr = np.asarray(green)
        green_array_1d = greenarr.flatten()
        sorted_arr = np.sort(green_array_1d)
        ll = len(sorted_arr)
        green_array = []
        for i in range(input_tsbtc):
            if i != input_tsbtc - 1:
                green_array.append(np.average(
                    sorted_arr[i*ll//input_tsbtc: (i+1)*ll//input_tsbtc]))
            else:
                green_array.append(np.average(sorted_arr[i*ll//input_tsbtc:]))
        for i in range(1, input_tsbtc+1):
            sheet1.write(count, input_tsbtc+i, green_array[i-1])
        # print(imagePath)

        # blue color
        bluearr = np.asarray(blue)
        blue_array_1d = bluearr.flatten()
        sorted_arr = np.sort(blue_array_1d)
        ll = len(sorted_arr)
        blue_array = []
        for i in range(input_tsbtc):
            if i != input_tsbtc - 1:
                blue_array.append(np.average(
                    sorted_arr[i*ll//input_tsbtc: (i+1)*ll//input_tsbtc]))
            else:
                blue_array.append(np.average(sorted_arr[i*ll//input_tsbtc:]))
        for i in range(1, input_tsbtc+1):
            sheet1.write(count, 2*input_tsbtc+i, blue_array[i-1])

        sheet1.write(count, 3*input_tsbtc+1, str(imagePath.split('.')[0]))
        count += 1
    print('Completed.....................')
    os.chdir(curr_dir)

write_into_word("D:\deepfake_detection\datasets\\main_test_imgs")


Completed.....................


In [13]:
file_path = f'D:\deepfake_detection\datasets\\new_densenet-121\\tsbtc{input_tsbtc}.xls'
wb.save(file_path)

- Convert to excel
- drop 1st col
- combine

In [14]:
tsbtc_df = pd.read_excel(file_path)
tsbtc_df.drop(['Unnamed: 0'], axis=1, inplace=True)
tsbtc_df['Images'] = tsbtc_df['Images'].apply(lambda x: x+'.jpg')
tsbtc_df.rename(columns = {'Images':'file_name'}, inplace = True)
df_reset = df.reset_index()
df_tsbtc = pd.merge(tsbtc_df, df_reset, on='file_name')
col = [x for x in df_tsbtc.columns if x not in ['file_name', 'label']]
df_tsbtc[col] = df_tsbtc[col].divide(100, axis="index")
df_tsbtc.to_csv(f'D:\deepfake_detection\datasets\\new_densenet-121\\labeled_tsbtc{input_tsbtc}.csv', index=False)
df_tsbtc.head(2)

,R1,R2,R3,R4,R5,R6,G1,G2,G3,G4,G5,G6,B1,B2,B3,B4,B5,B6,file_name,label
0,0.694032,1.047092,1.117619,1.149462,1.203579,1.461159,0.427495,0.626526,0.784330,1.527369,1.694441,1.733893,0.390574,0.550220,0.675696,1.284110,1.381104,1.412849,abarnvbtwb_1.jpg,1
1,0.655607,1.052183,1.139060,1.166205,1.269420,1.535244,0.413306,0.641935,0.862667,1.410843,1.687322,1.735606,0.384520,0.573646,0.803676,1.291675,1.384189,1.443684,abarnvbtwb_10.jpg,1


In [15]:
df = pd.read_csv(feature_file_name)
df.head(2)

,file_name,label,0,1,2,3,4,5,6,7,...,502,503,504,505,506,507,508,509,510,511
0,abarnvbtwb_10.jpg,1,0.012252,0.0,2.452108,0.725467,0.147997,0.0,0.307768,0.437981,...,0.000000,0.060367,0.281849,1.058227,0.0,0.198189,0.0,0.724373,0.0,0.0
1,abarnvbtwb_11.jpg,1,0.738508,0.0,1.667492,0.356716,0.063301,0.0,0.000000,0.000000,...,1.020212,0.000000,0.000000,3.055794,0.0,1.051151,0.0,0.532673,0.0,0.0


In [16]:
df_tsbtc.drop(['label'], inplace=True, axis=1)
df_final = pd.merge(df, df_tsbtc, on='file_name')
df_final.head(2)

,file_name,label,0,1,2,3,4,5,6,7,...,G3,G4,G5,G6,B1,B2,B3,B4,B5,B6
0,abarnvbtwb_10.jpg,1,0.012252,0.0,2.452108,0.725467,0.147997,0.0,0.307768,0.437981,...,0.862667,1.410843,1.687322,1.735606,0.384520,0.573646,0.803676,1.291675,1.384189,1.443684
1,abarnvbtwb_11.jpg,1,0.738508,0.0,1.667492,0.356716,0.063301,0.0,0.000000,0.000000,...,0.862629,1.405570,1.698157,1.745562,0.395478,0.583161,0.799501,1.305344,1.400231,1.460189


In [17]:
df_final.to_csv(f'D:\deepfake_detection\datasets\\new_densenet-121\densenet121_tsbtc{input_tsbtc}.csv', index=False)

# Machine Learning

## Densenet-121

In [18]:
from sklearn import svm
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from numpy import mean
from numpy import std
featureTsbtc = f"D:\deepfake_detection\datasets\\new_densenet-121\\densenet121_tsbtc{input_tsbtc}.csv"
onlyTsbtc = f"D:\deepfake_detection\datasets\\new_densenet-121\\labeled_tsbtc{input_tsbtc}.csv"


df_final = pd.read_csv(featureTsbtc) # Feature + tsbtc
# df_final = pd.read_csv(feature_file_name) # Only Features
# df_final = pd.read_csv(onlyTsbtc) # Only TSBTc

df_final.head(2)

,file_name,label,0,1,2,3,4,5,6,7,...,G3,G4,G5,G6,B1,B2,B3,B4,B5,B6
0,abarnvbtwb_10.jpg,1,0.012252,0.0,2.452108,0.725467,0.147997,0.0,0.307768,0.437981,...,0.862667,1.410843,1.687322,1.735606,0.384520,0.573646,0.803676,1.291675,1.384189,1.443684
1,abarnvbtwb_11.jpg,1,0.738508,0.0,1.667492,0.356716,0.063301,0.0,0.000000,0.000000,...,0.862629,1.405570,1.698157,1.745562,0.395478,0.583161,0.799501,1.305344,1.400231,1.460189


In [19]:
try:
    df_final.drop(['file_name'], axis=1, inplace=True)
except:
    pass

X = df_final.drop(['label'], axis=1)
y = df_final['label']

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)


In [20]:
X_test

,0,1,2,3,4,5,6,7,8,9,...,G3,G4,G5,G6,B1,B2,B3,B4,B5,B6
382,0.000000,0.000000,0.414835,0.000000,0.879111,0.0,0.000000,0.097788,0.0,0.000000,...,1.094196,1.272918,1.609072,2.173722,0.407635,0.588006,0.805816,1.020896,1.252266,1.815380
529,0.000000,0.221610,1.434429,0.476962,0.163864,0.0,0.000000,0.000000,0.0,0.000000,...,1.098040,1.599598,1.927734,2.012787,0.175872,0.392367,0.759597,1.197116,1.572234,1.640505
1231,0.720852,0.269156,0.926578,0.463991,0.377818,0.0,0.000000,0.000000,0.0,0.000000,...,0.556722,0.692226,0.911766,1.564380,0.198797,0.357891,0.495153,0.642247,0.835040,1.448996
211,0.315943,0.000000,1.801774,0.000000,0.000000,0.0,0.217571,0.000000,0.0,0.000000,...,0.770221,1.422023,1.699024,1.737955,0.377150,0.546061,0.668824,1.221235,1.381949,1.423770
1393,1.268371,0.000000,2.812920,0.000000,0.000000,0.0,0.000000,2.403211,0.0,0.000000,...,0.854787,1.005789,1.151936,1.458125,0.330723,0.541722,0.682645,0.865164,1.026041,1.322607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,0.000000,0.000000,1.583596,0.000000,0.000000,0.0,0.147320,0.448731,0.0,0.105273,...,0.517284,1.043475,1.722252,1.932850,0.078043,0.156721,0.248062,0.471912,1.093526,1.436823
1368,0.759478,0.000000,1.681615,0.311294,0.030573,0.0,0.000000,0.000000,0.0,0.000000,...,0.984477,1.537208,1.896910,2.326379,0.263610,0.612741,0.802395,1.449224,1.789125,2.336598
2122,0.282486,0.000000,0.829568,0.921919,0.410914,0.0,0.233484,0.000000,0.0,0.000000,...,0.386734,0.431657,0.481589,0.638431,0.125013,0.208702,0.257789,0.289718,0.340112,0.456031
141,0.148152,0.683863,0.867477,0.444672,0.153242,0.0,0.303271,1.008154,0.0,0.000000,...,1.021748,1.208100,1.568293,2.177922,0.361688,0.526303,0.744955,0.972474,1.244252,1.867954


In [21]:
y_test.value_counts()

1    341
0    317
Name: label, dtype: int64

### SVM

In [22]:
from sklearn.metrics import classification_report
clf = svm.SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred)) # TP FP FN TN
print(classification_report(y_test, y_pred, output_dict=True))

Accuracy: 0.5790273556231003
[[207 110]
 [167 174]]
{'0': {'precision': 0.553475935828877, 'recall': 0.6529968454258676, 'f1-score': 0.5991316931982633, 'support': 317}, '1': {'precision': 0.6126760563380281, 'recall': 0.5102639296187683, 'f1-score': 0.5568000000000001, 'support': 341}, 'accuracy': 0.5790273556231003, 'macro avg': {'precision': 0.5830759960834526, 'recall': 0.5816303875223179, 'f1-score': 0.5779658465991317, 'support': 658}, 'weighted avg': {'precision': 0.5841556335395465, 'recall': 0.5790273556231003, 'f1-score': 0.5771938400362454, 'support': 658}}


### Decision Tree Classifier

In [23]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred)) # TP FP FN TN
print(classification_report(y_test, y_pred, output_dict=True))


Accuracy: 0.8875379939209727
[[276  41]
 [ 33 308]]
{'0': {'precision': 0.8932038834951457, 'recall': 0.8706624605678234, 'f1-score': 0.8817891373801918, 'support': 317}, '1': {'precision': 0.8825214899713467, 'recall': 0.9032258064516129, 'f1-score': 0.8927536231884058, 'support': 341}, 'accuracy': 0.8875379939209727, 'macro avg': {'precision': 0.8878626867332462, 'recall': 0.8869441335097181, 'f1-score': 0.8872713802842989, 'support': 658}, 'weighted avg': {'precision': 0.8876678710458822, 'recall': 0.8875379939209727, 'f1-score': 0.8874713405118042, 'support': 658}}


### Random Forest

In [24]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred)) # TP FP FN TN
print(classification_report(y_test, y_pred, output_dict=True))

Accuracy: 0.6854103343465046
[[166 151]
 [ 56 285]]
{'0': {'precision': 0.7477477477477478, 'recall': 0.5236593059936908, 'f1-score': 0.6159554730983302, 'support': 317}, '1': {'precision': 0.6536697247706422, 'recall': 0.8357771260997068, 'f1-score': 0.7335907335907337, 'support': 341}, 'accuracy': 0.6854103343465046, 'macro avg': {'precision': 0.700708736259195, 'recall': 0.6797182160466988, 'f1-score': 0.6747731033445319, 'support': 658}, 'weighted avg': {'precision': 0.6989930276334727, 'recall': 0.6854103343465046, 'f1-score': 0.6769184272440895, 'support': 658}}


### Naive Bayes

In [25]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred)) # TP FP FN TN
print(classification_report(y_test, y_pred, output_dict=True))

Accuracy: 0.5577507598784195
[[199 118]
 [173 168]]
{'0': {'precision': 0.5349462365591398, 'recall': 0.6277602523659306, 'f1-score': 0.5776487663280117, 'support': 317}, '1': {'precision': 0.5874125874125874, 'recall': 0.49266862170087977, 'f1-score': 0.5358851674641147, 'support': 341}, 'accuracy': 0.5577507598784195, 'macro avg': {'precision': 0.5611794119858635, 'recall': 0.5602144370334052, 'f1-score': 0.5567669668960632, 'support': 658}, 'weighted avg': {'precision': 0.5621362451321271, 'recall': 0.5577507598784195, 'f1-score': 0.5560053207161745, 'support': 658}}


### Logistic Regression

In [26]:
from sklearn.linear_model import LogisticRegression  
clf = LogisticRegression()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred)) # TP FP FN TN
print(classification_report(y_test, y_pred, output_dict=True))

Accuracy: 0.7781155015197568
[[250  67]
 [ 79 262]]
{'0': {'precision': 0.7598784194528876, 'recall': 0.7886435331230284, 'f1-score': 0.7739938080495357, 'support': 317}, '1': {'precision': 0.7963525835866262, 'recall': 0.7683284457478006, 'f1-score': 0.782089552238806, 'support': 341}, 'accuracy': 0.7781155015197568, 'macro avg': {'precision': 0.778115501519757, 'recall': 0.7784859894354145, 'f1-score': 0.7780416801441709, 'support': 658}, 'weighted avg': {'precision': 0.7787806838443844, 'recall': 0.7781155015197568, 'f1-score': 0.7781893228953429, 'support': 658}}


c:\Users\iamsu\miniconda3\envs\data_sci\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### KNN

In [27]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred)) # TP FP FN TN
print(classification_report(y_test, y_pred, output_dict=True))

Accuracy: 0.5
[[158 159]
 [170 171]]
{'0': {'precision': 0.4817073170731707, 'recall': 0.49842271293375395, 'f1-score': 0.48992248062015503, 'support': 317}, '1': {'precision': 0.5181818181818182, 'recall': 0.501466275659824, 'f1-score': 0.5096870342771983, 'support': 341}, 'accuracy': 0.5, 'macro avg': {'precision': 0.49994456762749445, 'recall': 0.499944494296789, 'f1-score': 0.4998047574486767, 'support': 658}, 'weighted avg': {'precision': 0.500609756097561, 'recall': 0.5, 'f1-score': 0.5001652052357352, 'support': 658}}


### Ada Boost Classifier

In [28]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred)) # TP FP FN TN
print(classification_report(y_test, y_pred, output_dict=True))

Accuracy: 0.9559270516717325
[[301  16]
 [ 13 328]]
{'0': {'precision': 0.9585987261146497, 'recall': 0.9495268138801262, 'f1-score': 0.9540412044374009, 'support': 317}, '1': {'precision': 0.9534883720930233, 'recall': 0.9618768328445748, 'f1-score': 0.9576642335766424, 'support': 341}, 'accuracy': 0.9559270516717325, 'macro avg': {'precision': 0.9560435491038365, 'recall': 0.9557018233623504, 'f1-score': 0.9558527190070216, 'support': 658}, 'weighted avg': {'precision': 0.9559503511581533, 'recall': 0.9559270516717325, 'f1-score': 0.9559187924867647, 'support': 658}}


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
 
  
# creating the dataset
data = {'SVM':85.106, 'Random Forest':60.182, 'Decision Tree':84.954,
        'Naive Bayes':51.367, 'Logistic Regression': 78.115, 'KNN':97.872, 'AdaBoost':89.665}
courses = list(data.keys())
values = list(data.values())
  
fig = plt.figure(figsize = (15, 5))
 
# creating the bar plot
plt.bar(courses, values, color ='green', width = 0.5)
 
plt.xlabel("Classifiers")
plt.ylabel("Accuracy")
plt.title("Accuracies for Densenet")
plt.show()

## Inception